# Import libraries

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

#Lấy Link sách của tác giả cần craw data


In [14]:
url = 'https://www.goodreads.com/search?utf8=%E2%9C%93&query=Higashino+Keigo'

#Lấy Link các trang chứa file sách

In [ ]:
def link_page(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'lxml')
  link_page = soup.find_all('a', {'href': re.compile('\/search\?page=')})[:-1]
  link_page = ['https://www.goodreads.com'+ item['href'] for item in link_page]
  link_page = [url, *link_page]
  return link_page

In [ ]:
lk_page = link_page(url)

# Lấy link của từng sách 

In [ ]:
def link_book(link_page):
  link = []
  for i in link_page:
    url = i
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    link_book = soup.find_all('a', {'href': re.compile('\/book\/show/')})
    for item in link_book:
      link_book = 'https://www.goodreads.com'+ item.get('href')
      if link_book not in link:
        link.append(link_book)
  return link

In [ ]:
lk_book = link_book(lk_page)
lk_book

['https://www.goodreads.com/book/show/32058483-keigo-higashino-collection-box?from_search=true&from_srp=true&qid=DcLKcVrFIF&rank=1',
 'https://www.goodreads.com/book/show/44287550-the-miracles-of-the-namiya-general-store?from_search=true&from_srp=true&qid=DcLKcVrFIF&rank=2',
 'https://www.goodreads.com/book/show/8686068-the-devotion-of-suspect-x?from_search=true&from_srp=true&qid=DcLKcVrFIF&rank=3',
 'https://www.goodreads.com/book/show/13506866-salvation-of-a-saint?from_search=true&from_srp=true&qid=DcLKcVrFIF&rank=4',
 'https://www.goodreads.com/book/show/33602102-newcomer?from_search=true&from_srp=true&qid=DcLKcVrFIF&rank=5',
 'https://www.goodreads.com/book/show/20613611-malice?from_search=true&from_srp=true&qid=DcLKcVrFIF&rank=6',
 'https://www.goodreads.com/book/show/28823155-the-name-of-the-game-is-a-kidnapping?from_search=true&from_srp=true&qid=DcLKcVrFIF&rank=7',
 'https://www.goodreads.com/book/show/19256975-journey-under-the-midnight-sun?from_search=true&from_srp=true&qid=Dc

#Lấy tên toàn bộ sách của tác giả

In [ ]:
def name_book(link_page):
  name_book = []
  for i in link_page:
    url = i
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    book = soup.find_all('a', class_="bookTitle")
    book = [item.text.replace('\n','').strip() for item in book]
    name_book.extend(book)
  return name_book

In [ ]:
namebook = name_book(lk_page)

# Lấy rating và số người rate

In [ ]:
def rating_personrate(link_page):
  rating_vote = []
  num_person_vote = []
  for i in link_page:
    url = i
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    rating = soup.find_all('span', class_ ="minirating")
    # rating_vote.extend([rating.text.split(' ')[1]])
    rating = [item.text for item in rating]
    rating_vote.extend([rating[j].split(' ')[1] for j in range(len(rating))])
    num_person_vote.extend([rating[j].split(' ')[5] for j in range(len(rating))])
  return rating_vote, num_person_vote

In [ ]:
rate_vote, numperson_vote = rating_personrate(lk_page)

In [ ]:
len(rate_vote)

131

#Lấy ảnh bìa và tên tác giả


In [ ]:
def anhbia(link_page):
  img_link = []
  for i in link_page:
    url = i
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    image = soup.find_all('img', itemprop ="image")
    img_link.extend([item['src'] for item in image])
  return img_link


In [ ]:
anh_bia = anhbia(lk_page)

In [ ]:
def authorname(link_page):
  author_name = []
  for i in link_page:
    url = i
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    author = soup.find_all('span', itemprop="author")
    author_name.append([item.text.replace('\n','').strip().replace('(Translator)','').strip() for item in author])
    name_author = []
    for i in range(len(author_name)):
      name_author += author_name[i]
  return name_author

In [ ]:
author_name = authorname(lk_page)

#Lấy Tóm tắt và số người review

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 5.1 MB/s 
     |████████████████████████████████| 358 kB 45.0 MB/s 
     |████████████████████████████████| 138 kB 72.6 MB/s 
     |████████████████████████████████| 4.1 MB 46.9 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
def tomtat_sach(link):
  tomtat = []
  person_review = []
  for i in link:
    url = i
    wd.get(url)
    review = wd.find_element(By.XPATH, "//*[@id='bookMeta']/a[3]")
    person_review.extend([review.text])
    try:
      a = wd.find_element(By.ID, 'description')
      more = wd.find_element(By.XPATH, "//*[@id='description']/a")
      more.send_keys(Keys.RETURN)
      b = wd.find_element(By.ID, 'description')
      tomtat.extend([b.text])
      review = wd.find_element(By.XPATH, "//*[@id='bookMeta']/a[3]")
    except:
      try:
        b = wd.find_element(By.ID, 'description')
        tomtat.extend([b.text])
      except:
        tomtat.extend(["None"])
  return tomtat, person_review

In [ ]:
tomtat, person_review = tomtat_sach(lk_book)


#Bình luân

In [15]:
def lay_binhluan(link):
  binhluan_link = []
  for i in lk_book:
    url = i
    wd.get(url)
    try:
      binhluan = wd.find_elements(By.XPATH, "//a[text() = 'see review']")
      binhluan_link.extend([item.get_attribute('href') for item in binhluan])
      page = wd.find_element(By.CLASS_NAME, "containerWithHeaderContent").find_element(By.XPATH, "//*[@id='reviews']/div[5]").find_element(By.CLASS_NAME, "next_page")
      page.send_keys(Keys.RETURN)
      time.sleep(2)
      binhluan = wd.find_elements(By.XPATH, "//a[text() = 'see review']")
      binhluan_link.extend([item.get_attribute('href') for item in binhluan])
    except:
      pass
  return binhluan_link

In [16]:
binhluan_link = lay_binhluan(lk_book)

In [17]:
len(binhluan_link)

3328

In [18]:
def value_binhluan(link):
  value = []
  url = link
  wd.get(url)
  try:
    value.extend([wd.find_element(By.CLASS_NAME, "hreview").find_element(By.CLASS_NAME, "bookTitle").text])
  except:
    value.extend(["None"])
  try:
    value.extend([wd.find_element(By.CLASS_NAME, "reviewer").find_element(By.CLASS_NAME, "userReview").text])
  except:
    value.extend(["None"])
  try:
    value.extend([wd.find_element(By.CLASS_NAME, "dtreviewed ").text])
  except:
    value.extend(["None"])
  try:
    value.extend([wd.find_element(By.CLASS_NAME, "description").text])
  except:
    value.extend(["None"])
  try:
    value.extend([wd.find_element(By.CLASS_NAME, "likesCount").text])
  except:
    value.extend(["0 like"])
  try:
    value.extend([wd.find_element(By.CLASS_NAME, "rating").find_element(By.TAG_NAME, 'meta').get_attribute("content")])
  except:
    value.extend(["0"])
  # value = [tacgia, noi dung, so like, ..., [(ten, ngay, ndd), (ten, ngay, ndd)]]
  # value.extend

  return value


In [ ]:
value = value_binhluan(binhluan_link[0])

In [ ]:
value

['Keigo Higashino Collection Box: The Devotion of Suspect X, Salvation of a Saint, Malice',
 'Joanne',
 'Aug 23, 2018',
 'Brilliant, it’s been awhile since I gave a 5⭐️ Rating but this book is deservingly so. Fabulous, had me hook from start to finish. I loved the approach by the author. This isn’t your typical who done it. From the start you know who and how but keep reading cause it’s just the tip,of the iceberg and all will be revealed in the most unexpected way, totally blew me away. 👏👏👏👏👏👏👍👍👍😎 btw the ending —HEARTBREAKING!',
 '10 likes',
 '5']

In [19]:
def value_comment(link):
  value = []   #ten,ngay,nd
  date = []
  name_comment = []
  nd = []
  url = link
  wd.get(url)
  try:
    name = wd.find_elements(By.CLASS_NAME, "commentAuthor")
    name_comment.extend([item.text for item in name])
  except:
    name_comment.extend(["None"])
  try:
    day = wd.find_elements(By.CLASS_NAME, "brownBox")
    date.extend([item.find_element(By.CLASS_NAME, "right").text for item in day])
  except:
    date.extend(["None"])
  try:
    noidung = wd.find_elements(By.CLASS_NAME, "comment")
    nd.extend([item.find_element(By.CLASS_NAME, "mediumText").text for item in noidung])
  except:
    nd.extend(["None"])
  try:
    for i in range(len(name_comment)):
      value.append(tuple((name_comment[i], date[i], nd[i])))
  except:
    pass
  return value

In [20]:
def loop_binhluan(link_all):
  values = []
  j = 0
  for i in link_all:
    values.append(value_binhluan(i))
    time.sleep(0.5)
    values[j].append(value_comment(i))
    j += 1
    print(j)
  return values


In [22]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
values = loop_binhluan(binhluan_link)

In [23]:
values[0]

['Keigo Higashino Collection Box: The Devotion of Suspect X, Salvation of a Saint, Malice',
 'Joanne',
 'Aug 23, 2018',
 'Brilliant, it’s been awhile since I gave a 5⭐️ Rating but this book is deservingly so. Fabulous, had me hook from start to finish. I loved the approach by the author. This isn’t your typical who done it. From the start you know who and how but keep reading cause it’s just the tip,of the iceberg and all will be revealed in the most unexpected way, totally blew me away. 👏👏👏👏👏👏👍👍👍😎 btw the ending —HEARTBREAKING!',
 '10 likes',
 '5',
 []]

In [26]:
from IPython.display import display
pd.set_option('max_colwidth', 20000)
pd.set_option('precision', 5)
pd.set_option('max_rows', 10000)
pd.set_option('max_columns', 10000)

df = pd.DataFrame(values, columns=['Tên sách', 'Tên người bình luận', 'Ngày bình luận','Nội dung','số sao','số like',' comment'])

In [27]:
df.to_csv('/content/drive/MyDrive/craw_data/data_comment.csv')

#Lưu file .csv

In [ ]:
from IPython.display import display
pd.set_option('max_colwidth', 200)
pd.set_option('precision', 5)
pd.set_option('max_rows', 150)
pd.set_option('max_columns', 10)

dict_columns = {
    'tên sách': name_book,
    'số sao đánh giá': rate_vote,
    'số người đánh giá': numperson_vote,
    'link ảnh bìa': anh_bia,
    'tên tác giả': author_name,
    'tom tat': tomtat,
    'so nguoi review': person_review
}

df = pd.DataFrame(dict_columns)
display(df)

,tên sách,số sao đánh giá,số người đánh giá,link ảnh bìa,tên tác giả,tom tat,so nguoi review
0,<function name_book at 0x7f1908796050>,4.42,183,https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1636180355i/32058483._SX50_.jpg,Keigo Higashino,None,18 reviews
1,<function name_book at 0x7f1908796050>,4.45,"17,389",https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1564085721i/44287550._SX50_.jpg,"Keigo Higashino, Sam Bett","When three delinquents hole up in an abandoned general store after their most recent robbery, to their great surprise, a letter drops through the mail slot in the store's shutter. This seemingly s...","3,309 reviews"
2,<function name_book at 0x7f1908796050>,4.15,"46,350",https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1312051518i/8686068._SY75_.jpg,Keigo Higashino,"Yasuko lives a quiet life, working in a Tokyo bento shop, a good mother to her only child. But when her ex-husband appears at her door without warning one day, her comfortable world is shattered.\...","6,272 reviews"
3,<function name_book at 0x7f1908796050>,3.98,"15,732",https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1495618717i/13506866._SX50_.jpg,"Keigo Higashino, Alexander O. Smith","In 2011, The Devotion of Suspect X was a hit with critics and readers alike. The first major English language publication from the most popular bestselling writer in Japan, it was acclaimed as “st...","2,127 reviews"
4,<function name_book at 0x7f1908796050>,4.01,"17,517",https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1497270264i/20613611._SY75_.jpg,"Keigo Higashino, Alexander O. Smith","Acclaimed bestselling novelist Kunihiko Hidaka is found brutally murdered in his home on the night before he's planning to leave Japan and relocate to Vancouver. His body is found in his office, a...","2,765 reviews"
5,<function name_book at 0x7f1908796050>,3.75,"2,252",https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1488294138i/28823155._SX50_.jpg,Keigo Higashino,Sakuma is a high-profile ad agent who was about to land one of the biggest gigs of his career. But he was betrayed by the owner of the company that just hired him.\n\nDown on his luck and now on h...,411 reviews
6,<function name_book at 0x7f1908796050>,4.33,"12,130",https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1386409101i/19256975._SY75_.jpg,"Keigo Higashino, Alexander O. Smith","When a man is found murdered in an abandoned building in Osaka in 1973, unflappable detective Sasagaki is assigned to the case. He begins to piece together the connection of two young people who a...","1,720 reviews"
7,<function name_book at 0x7f1908796050>,4.05,"6,655",https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1530346547i/33602102._SY75_.jpg,"Keigo Higashino, Giles Murray","Detective Kyoichiro Kaga of the Tokyo Police Department has just been transferred to a new precinct in the Nihonbashi area of Tokyo. Newly arrived, but with a great deal of experience, Kaga is pro...","1,200 reviews"
8,<function name_book at 0x7f1908796050>,4.57,496,https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1341936865i/15745357._SX50_.jpg,"Keigo Higashino, 東野圭吾, 劉姿君",Traditional Chinese first edition can be found HERE\n\n「在我們的頭上沒有太陽，一直都是夜晚，\n但是卻不覺得黑暗，因為有能代替太陽的人在。」\n\n他們為什麼如此輕易地抹煞別人的性命？\n他們為什麼毫無愧疚地奪取別人的靈魂？\n\n一九七三年，大阪的廢棄大樓發現了一具他殺屍體，被害者之子桐原亮司與嫌疑犯之女西本雪穗，就此走上截然不同的...,24 reviews
9,<function name_book at 0x7f1908796050>,4.13,"1,290",https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1573913458i/48837691._SY75_.jpg,"Keigo Higashino, Thu Hiền","Tác phẩm là một áng văn trần trụi ngột ngạt ứa ra từ người tù sau song sắt.\n\nLâu nay, nếu có giai thoại gì về người tù, thì đó thường là một người tù kiên trì, có thành tựu.\n\nVí như kiên trì r...",264 reviews
